<a href="https://colab.research.google.com/github/camismg/TCC---Prof.Bruno/blob/main/GSI073_aula0_regressao_logistica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GSI073 - Tópicos Especiais de Inteligência Artificial

## Definição dos dados

In [ ]:
import torch; import sklearn

# 1. Carregar dados
iris = sklearn.datasets.load_iris()
X = iris.data        # 4 features: sépalas e pétalas
y = (iris.target == 1).astype(float)  # 1 se Versicolor, 0 caso contrário

# 2. Preparar dados para pytorch
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32).view(-1, 1)

tmp=y
print(tmp)
print(tmp.shape)

In [ ]:
import sklearn.datasets
from sklearn.model_selection import train_test_split
import torch

# 1. Carregar dados
iris = sklearn.datasets.load_iris()
X = iris.data        # 4 features: sépalas e pétalas
y = (iris.target == 1).astype(float)  # 1 se Versicolor, 0 caso contrário

# 2. Separar conjunto de treino e teste (80% treino, 20% teste)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 3. Preparar dados para PyTorch
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test  = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
y_test  = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# 4. Teste: imprimir shapes e alguns valores
print("Conjunto de Treino")
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("Primeiros rótulos do treino:\n", y_train[:10])

print("\nConjunto de Teste")
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)
print("Primeiros rótulos do teste:\n", y_test[:10])


## Definição do modelo e treinamento

In [ ]:
# 3. Definir modelo: regressão logística
modelo = torch.nn.Linear(4, 1)  # 4 features → 1 saída (probabilidade de ser Versicolor)

# 4. Definir função de perda e algoritmo de otimização
funcao_perda = torch.nn.BCEWithLogitsLoss()  # combinação de sigmoid + BCE
optimizer = torch.optim.SGD(modelo.parameters(), lr=0.1)

print("Dados")
print("Treino:", X_train.shape, y_train.shape)
print("Teste:", X_test.shape, y_test.shape)

print("\nModelo")
print(modelo)

## Execução do treinamento

In [ ]:
# 5. Treino
for epoch in range(1000):
    optimizer.zero_grad() # reseta gradiente senão acumula
    outputs = modelo(X_train)
    loss = funcao_perda(outputs, y_train)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Época [{epoch+1}/100], Loss: {loss.item():.4f}")

    # 6. Treinamento (apenas com dados de treino)
for epoch in range(1000):
    optimizer.zero_grad()                 # zera gradientes
    outputs = modelo(X_train)             # predições no treino
    loss = funcao_perda(outputs, y_train) # calcula erro
    loss.backward()                       # retropropagação
    optimizer.step()                      # atualização dos pesos

    if (epoch + 1) % 100 == 0:
        print(f"Época [{epoch+1}/1000] - Loss: {loss.item():.4f}")

# 7. Validação no conjunto de teste
with torch.no_grad():  # sem calcular gradientes
    outputs_test = modelo(X_test)
    probs = torch.sigmoid(outputs_test)      # converte logits em probabilidades
    preds = (probs >= 0.5).float()           # 1 se prob >= 0.5, senão 0
    corretos = preds == y_test               # compara predição e rótulo
    acuracia = corretos.float().mean().item() * 100

# 8. Mostrar resultados
print("\nResultados")
print("Predições (True = acerto, False = erro):\n", corretos.flatten())
print(f"Acurácia no conjunto de teste: {acuracia:.2f}%")


